## ETL

In [1]:
# Librerias necesarias
import pandas as pd
import numpy as np
import json
import ast
import gzip

C:\Users\Martin\AppData\Local\Temp\ipykernel_2240\4171661857.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Ruta de los datos:
ruta_games= 'Datasets originales/steam_games.json.gz'
ruta_user_reviews= 'Datasets originales/user_reviews.json.gz'
ruta_user_items= 'Datasets originales/users_items.json.gz'

#### Games

In [3]:
## Games:
games_orig = []
with gzip.open(ruta_games, 'rt', encoding='utf-8') as archivo_comprimido:
    for linea in archivo_comprimido:
        diccionario = json.loads(linea)
        games_orig.append(diccionario)

# Transformamos a DataFrame:
games_orig = pd.DataFrame(games_orig)

In [4]:
# Copia de los datos para no estar constantemente cargandolos para resetearlos
games = games_orig.copy()
games.shape

(120445, 13)

In [5]:
games.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


Las transformaciones que realizaremos en este caso serán: 

1. Eliminar registros completamente vacíos. 
2. Eliminar columnas que no utilizaremos: ``publisher``, ``url``, ``reviews_url``, ``price``, ``early_access``, ``developer``. Eliminar la columna ``tags`` previo a rellenar los datos faltantes de genres con sus valores. ``app_name`` y ``title`` dicen lo mismo, dejaremos la columna que tenga menor cantidad de datos faltantes.
3. Desanidar registros que poseen valores con tipo de dato LISTA.
4. Eliminar registros duplicados.
5. Corregir el tipo de dato de cada columna. Los registros que poseen el campo ``release_date`` vacío o con un dato que no corresponde, se rellenaran con '1900-01-01' para poder mantener el registro sin perjudicar el análisis. También creamos ``release_year``.
6. Eliminar registros que poseen datos vacíos en columnas relevantes.
7. Renombraremos la columna ``id`` a ``ìtem_id``.

In [6]:
# 1. Eliminar registros completamente vacíos
print(f'tamaño inicial del dataframe: {len(games)} registros')
games.dropna(how='all', inplace= True, ignore_index=True)
print(f'tamaño final del dataframe: {len(games)} registros ')

tamaño inicial del dataframe: 120445 registros
tamaño final del dataframe: 32135 registros 


In [7]:
# 2. Eliminar columnas que no utilizaremos: ``publisher``, ``url``, ``reviews_url``, ``price``, ``early_access``, ``developer``.
# Eliminar la columna ``tags`` previo a rellenar los datos faltantes de ``genres`` con sus valores. 
# ``app_name`` y ``title`` dicen lo mismo, dejaremos "app_name" que tiene menos datos faltantes y le cambiaremos el nombre a "title"

print(f"El tamaño inicial de games era {games.shape}")

games['genres'] = games['genres'].combine_first(games['tags']) # a cada valor faltante de genres le hara corresponder lo que figure en tags
games['title']= games['app_name']
games.drop(columns=['publisher', 'url', 'reviews_url', 'price', 'early_access', 'developer', 'tags', 'app_name'], inplace=True)

print("Columnas sin utilidad eliminadas")
print(f"Ahora, el tamaño final es: {games.shape}")


El tamaño inicial de games era (32135, 13)
Columnas sin utilidad eliminadas
Ahora, el tamaño final es: (32135, 5)


In [8]:
games.sample()

,genres,title,release_date,specs,id
28270,"[Adventure, Indie]",A Golden Wake,2014-10-09,"[Single-player, Steam Achievements, Steam Trad...",307570


In [9]:
# 3. Desanidar registros que poseen valores con tipo de dato LISTA: genres y specs.
games = games.explode(column=['genres'], ignore_index=True)
games = games.explode(column=['specs'], ignore_index=True)
games.shape

(408844, 5)

In [10]:
# 4. Eliminar registros duplicados.
games.drop_duplicates(inplace=True, ignore_index=True)
games.shape

(408840, 5)

In [11]:
# 5. Corregir el tipo de dato de cada columna.
## genres OK
## specs OK
## title OK
## release_date INCONSISTENTE
## id VALORES NAN. Borrarlos, no me sirven xq no puedo identificar sus reviews
fecha_relleno = pd.to_datetime('1900-01-01')
games['release_date'] = pd.to_datetime(games['release_date'], format='%Y-%m-%d', errors='coerce').fillna(fecha_relleno).dt.year
games.rename(columns={'release_date':'release_year','id':'item_id'}, inplace=True)
# Si no tiene ID no sirve, se dropea.
games.dropna(subset='item_id', inplace=True)
games['item_id']=games['item_id'].astype(int)
games.info()
games.shape

<class 'pandas.core.frame.DataFrame'>
Index: 408829 entries, 0 to 408839
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   genres        408631 non-null  object
 1   title         408825 non-null  object
 2   release_year  408829 non-null  int32 
 3   specs         407184 non-null  object
 4   item_id       408829 non-null  int32 
dtypes: int32(2), object(3)
memory usage: 15.6+ MB


(408829, 5)

In [12]:
# 6. Eliminar registros que poseen datos vacíos en columnas relevantes
item_id_sin_utilidad = games['item_id'][games['genres'].isna()].unique()
games.dropna(subset='genres', inplace=True, ignore_index=True)
games.shape

(408631, 5)

In [78]:
games_id = games[['item_id', 'title']].drop_duplicates()
games_id

,item_id,title
0,761140,Lost Summoner Kitty
5,643980,Ironbound
33,670290,Real Pool 3D - Poolians
58,767400,弹炸人2222
61,773570,Log Challenge
...,...,...
408545,773640,Colony On Mars
408553,733530,LOGistICAL: South Africa
408568,610660,Russian Roads
408577,658870,EXIT 2 - Directions


#### Para poder MERGEAR las tablas, los campos de las PK y FK deben ser el mismo tipo de dato. No puedo mergear un ``item_id`` string con un ``item_id`` integer

#### Reviews

In [13]:
## Reviews:
user_reviews = []
with gzip.open(ruta_user_reviews, 'rt', encoding="utf-8") as archivo_comprimido:
    for linea in archivo_comprimido:
        user_reviews.append(ast.literal_eval(linea))
    
# Transformamos a DataFrame:
user_reviews_orig = pd.DataFrame(user_reviews)


In [14]:
# Copia de los datos para no estar constantemente cargandolos para resetearlos
user_reviews = user_reviews_orig.copy()

In [ ]:
user_reviews.head()

In [15]:
# Extraigo solo las columnas que voy a usar

reviews = user_reviews[['user_id','reviews']]

In [16]:
# Desanidamos reviews:

reviews_explode = reviews.explode('reviews', ignore_index=True) # separo en filas todos todos los json que estan listados en cada registro
reviews_desanidado = pd.json_normalize(reviews_explode['reviews']) # transformo a tabla cada uno de los json que tenemos en cada registro
reviews = pd.concat([reviews_explode,reviews_desanidado], axis=1).drop(columns=['reviews']) # concateno ambos dataframes y dropeo la columna reviews que está anidada

In [ ]:
reviews.head()

In [17]:
reviews.shape

(59333, 8)

Las transformaciones que realizaremos en este caso serán: 

1. Removeremos los registros en el que la mayoria de los campos son NaN.
2. Removeremos la columna ``funny``, no nos interesa saber si una opinion es divertida, mas bien nos interesa si da una buena, mala o neutral opinion del juego. Tampoco nos interesa la última edicion del review, entonces ``last_edited`` será eliminada.
3. Corrección de la fecha de publicación del review, cambio del tipo de dato y renombramiento de ```posted`` a ``posted_year``.
4. Remover los registros es los que ``item_id`` coincide con **item_id_sin_utilidad**. Corresponden a ids de juegos que ya fueron borrados de nuestra BBDD.
5. Cambio del tipo de dato de la columna ``recommend``.


In [18]:
reviews[reviews['funny'].isna()].shape # todos estos son registros practicamente nulos

(28, 8)

In [19]:
# 1. Removeremos los registros en el que la mayoria de los campos son NaN.
reviews.dropna(subset='funny', inplace=True, ignore_index=True)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   funny        59305 non-null  object
 2   posted       59305 non-null  object
 3   last_edited  59305 non-null  object
 4   item_id      59305 non-null  object
 5   helpful      59305 non-null  object
 6   recommend    59305 non-null  object
 7   review       59305 non-null  object
dtypes: object(8)
memory usage: 3.6+ MB


In [20]:
# 2. Removeremos la columna ``funny``, no nos interesa saber si una opinion es divertida,
# mas bien nos interesa si da una buena, mala o neutral opinion del juego.
reviews.drop(columns=['funny', 'last_edited'], inplace=True)
reviews.sample()

,user_id,posted,item_id,helpful,recommend,review
56034,76561198089436384,"Posted November 21, 2015.",273110,1 of 3 people (33%) found this review helpful,False,esse jogo e tao lixo q parece q eu n esstava j...


In [21]:
# 3. Corrección de la fecha de publicación del review, cambio del tipo de dato
a_eliminar = reviews['posted'].str.contains(r'Posted [A-Z][a-z]+ \d{1,2}\.')
reviews=reviews[~a_eliminar]
reviews.reset_index(drop=True, inplace=True)
def extraer_año(texto):
    return texto[-5:-1]
reviews['posted'] = reviews['posted'].apply(lambda x: extraer_año(x)).astype('int32')
reviews.rename(columns={'posted':'posted_year'}, inplace=True)


In [22]:
# 4. Remover los registros es los que ``item_id`` coincide con **item_id_sin_utilidad**. Corresponden a ids de juegos que ya fueron borrados de nuestra BBDD.
reviews['item_id'] = reviews['item_id'].astype('int32')
reviews = reviews[~reviews['item_id'].isin(item_id_sin_utilidad)].reset_index(drop=True)
# solo se borró un registro, pero bueno, cualquier limpieza por mas chica que sea es bienvenida. 

In [23]:
reviews.shape

(49185, 6)

In [24]:
reviews['recommend'] = reviews['recommend'].astype('bool') 

#### Items

In [25]:
## Items:
user_items = []
with gzip.open(ruta_user_items, 'rt', encoding="utf-8") as archivo_comprimido:
    for linea in archivo_comprimido:
        user_items.append(ast.literal_eval(linea))
    
# Transformamos a DataFrame:
user_items_orig = pd.DataFrame(user_items)

In [26]:
# Copia de los datos para no estar constantemente cargandolos para resetearlos
user_items = user_items_orig.copy()

In [27]:
# Extraigo solo las columnas que voy a usar:
items = user_items[['user_id','items']]
items.head()

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [28]:
# Desanidamos items:

items_explode = items.explode('items', ignore_index=True) # separo en filas todos todos los json que estan listados en cada registro
items_desanidado = pd.json_normalize(items_explode['items']) # transformo a tabla cada uno de los json que tenemos en cada registro
items = pd.concat([items_explode,items_desanidado], axis=1).drop(columns=['items']) # concateno ambos dataframes y dropeo la columna items que está anidada

In [29]:
items.shape

(5170015, 5)

In [314]:
items.head()

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0


Las transformaciones que realizaremos en este caso serán: 

1. Removeremos los registros en el que la mayoria de los campos son NaN.

In [30]:
items.shape
items.dropna(subset='item_id', inplace=True, ignore_index=True)
items.shape

(5153209, 5)

#### Users_Recommend

In [132]:
# esto ira en el etl
df_user_recommend = reviews[['item_id','recommend', 'posted_year']]
df_user_recommend = df_user_recommend.merge(games_id, 'left', left_on='item_id', right_on='item_id')
df_user_recommend.to_csv('Datasets para API\df_user_recommend.csv', index=False)

#cond1 = df_user_recommend['recommend']==True
#cond2 = df_user_recommend['posted_year']==2015
#
#ep3 = df_user_recommend[cond1 & cond2].groupby('title').count().sort_values('recommend', ascending=False)
#top3 = ep3.head(3).index.tolist()

df_user_recommend = pd.read_csv(r'Datasets para API\df_user_recommend.csv')

def UsersRecommend(anio: int):
    cond1 = df_user_recommend['recommend']==True
    cond2 = df_user_recommend['posted_year']==anio
    ep3 = df_user_recommend[cond1 & cond2].groupby('title').count().sort_values('recommend', ascending=False)
    top3 = ep3.head(3).index.tolist()
    return [{'Puesto 1':top3[0]}, {'Puesto 2':top3[1]}, {'Puesto 3':top3[2]}]


def UsersNotRecommend(anio: int):
    cond1 = df_user_recommend['recommend']==False
    cond2 = df_user_recommend['posted_year']==anio
    ep4 = df_user_recommend[cond1 & cond2].groupby('title').count().sort_values('recommend', ascending=False)
    top3 = ep4.head(3).index.tolist()
    return [{'Puesto 1':top3[0]}, {'Puesto 2':top3[1]}, {'Puesto 3':top3[2]}]

In [134]:
UsersRecommend(2015)
# UsersNotRecommend(2015)

[{'Puesto 1': 'Counter-Strike: Global Offensive'},
 {'Puesto 2': 'Team Fortress 2'},
 {'Puesto 3': "Garry's Mod"}]

In [ ]:
df_user_recommend[['item_id','recommend']][(df_user_recommend['recommend']==True) & (df_user_recommend['posted_year']==year)].groupby('item_id').count().sort_values('recommend', ascending=False)
